# Redes Neuronales Recurrentes: Generación de texto con LSTMs

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico se propone al alumno la implementación de un programa que permita generar el titular de un artículo en el idioma Español.

Para ello, debe implementarse un modelo generativo utilizando Redes Neuronales Recurrentes (LSTMs).
</div>

<div style="background-color:#EEEEEE;color:black;padding:2%;">
<h2> Conjunto de datos </h2>
    
El conjunto de datos que debe utilizarse para este caso práctico esta compuesto por 1079 titulares escritos en español.
</div>

# Resolución del caso práctico

## 1. Lectura del conjunto de datos

In [ ]:
with open("/content/titulares.txt", encoding="latin-1") as f:
    titulares = f.read().splitlines()

In [ ]:
titulares[:10]

['Nueva ley de energía promete revolucionar el sector eléctrico',
 'El cambio climático sigue siendo una amenaza global',
 'Inversionistas buscan oportunidades en energías renovables',
 'Aumenta la demanda de vehículos eléctricos',
 'Vacunas contra COVID-19: ¿Cuándo estaremos todos protegidos?',
 'El debate sobre las vacunas sigue dividiendo opiniones',
 'Expertos en salud analizan la efectividad de las vacunas',
 'Vacunación masiva contra el coronavirus en marcha',
 'El mercado de criptomonedas se dispara a nuevas alturas',
 '¿Es Bitcoin la moneda del futuro?']

## 2. Preparación del conjunto de datos

### 2.1. Limpieza del conjunto de datos

In [ ]:
import string
import unicodedata

def preproc_texto(txt):
    txt = "".join(c for c in txt if c not in string.punctuation).lower()
    txt = unicodedata.normalize('NFKD', txt).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return txt

In [ ]:
dataset = [preproc_texto(t) for t in titulares]

In [ ]:
dataset[:10]

['nueva ley de energia promete revolucionar el sector electrico',
 'el cambio climatico sigue siendo una amenaza global',
 'inversionistas buscan oportunidades en energias renovables',
 'aumenta la demanda de vehiculos electricos',
 'vacunas contra covid19 cuando estaremos todos protegidos',
 'el debate sobre las vacunas sigue dividiendo opiniones',
 'expertos en salud analizan la efectividad de las vacunas',
 'vacunacion masiva contra el coronavirus en marcha',
 'el mercado de criptomonedas se dispara a nuevas alturas',
 'es bitcoin la moneda del futuro']

### 2.2. Tokenización

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
def tokenizar_texto(tokenizer, dataset):
  # Construccion del tokenizador
  tokenizer.fit_on_texts(dataset)
  total_palabras = len(tokenizer.word_index) + 1
  # Tokenizacion del texto del conjunto de datos
  dataset_tokens = []
  for text in dataset:
      titular_tokens = tokenizer.texts_to_sequences([text])[0]
      # Transformación del texto en ngramas
      for i in range(1, len(titular_tokens)):
          sec_n_gram = titular_tokens[:i+1]
          dataset_tokens.append(sec_n_gram)
  return dataset_tokens, total_palabras

In [ ]:
dataset_tokens, total_palabras = tokenizar_texto(tokenizer, dataset)

### 2.3. Generación de las etiquetas para el conjunto de datos

Cuando entrenamos una red neuronal recurrente (RNN), como una LSTM, para predecir palabras en un texto, estamos abordando una tarea de procesamiento de lenguaje natural (NLP) conocida como "modelado de lenguaje". El objetivo principal es que la red pueda aprender a predecir la próxima palabra en una secuencia de texto dada una serie de palabras anteriores.

Para entrenar esta red, necesitamos etiquetar nuestro conjunto de datos de entrenamiento de manera adecuada. En lugar de convertir directamente el texto en n-gramas, lo que hacemos es crear pares de entrada y salida. Aquí está cómo funciona con un ejemplo:

Ejemplo:

Supongamos que tenemos la siguiente frase:


```
El gato está en el tejado.
```


Creamos pares de entrada y salida de la siguiente manera:

```
Entrada: "El", Salida: "gato"
Entrada: "El gato", Salida: "está"
Entrada: "El gato está", Salida: "en"
Entrada: "El gato está en", Salida: "el"
Entrada: "El gato está en el", Salida: "tejado"
```

Cada "Entrada" es una secuencia de palabras en orden y la "Salida" es la siguiente palabra en esa secuencia. De esta manera, le estamos diciendo a la red que, dado un contexto previo de palabras, intente predecir cuál será la siguiente palabra. Esto permite que el modelo capture patrones y relaciones entre las palabras en función de su contexto.

Es importante señalar que, a pesar de que estamos trabajando con secuencias de palabras, estamos manteniendo el orden de las palabras para capturar el contexto y la dependencia entre ellas.

Una vez que tengas estos pares de entrada y salida etiquetados, puedes utilizarlos para entrenar una red neuronal, como una LSTM, para que pueda predecir la siguiente palabra en una secuencia de texto en función del contexto anterior. Esta es una técnica común en el procesamiento de lenguaje natural y se utiliza en aplicaciones como la generación de texto automático, corrección gramatical y muchas otras tareas relacionadas con el lenguaje natural.

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku

In [ ]:
def generación_etiquetas(dataset_tokens, total_palabras):
    # Obtengo el tamaño de la mayor secuencia del conjunto de datos
    max_sequence_len = max([len(text) for text in dataset_tokens])
    # Aplico padding al resto de secuencias hasta el tamaño máximo
    dataset_tokens = np.array(pad_sequences(dataset_tokens, maxlen=max_sequence_len, padding='pre'))
    # Genero las etiquetas para cada ejemplo del conjunto de datos
    X_train, y_train = dataset_tokens[:,:-1],dataset_tokens[:,-1]
    y_train = ku.to_categorical(y_train, num_classes=total_palabras)
    return X_train, y_train, max_sequence_len

In [ ]:
X_train, y_train, max_sequence_len = generación_etiquetas(dataset_tokens, total_palabras)

## 3. Construcción del modelo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
def create_model(max_sequence_len, total_palabras):
    input_len = max_sequence_len - 1
    model = Sequential()

    # Add Input Embedding Layer
    model.add(Embedding(total_palabras, 10, input_length=input_len))

    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))

    # Add Output Layer
    model.add(Dense(total_palabras, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

model = create_model(max_sequence_len, total_palabras)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 14, 10)            12170     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1217)              122917    
                                                                 
Total params: 179487 (701.12 KB)
Trainable params: 179487 (701.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

## 4. Predicción de nuevos ejemplos

In [ ]:
def generar_texto(prompt, num_palabras, modelo, max_sequence_len):
    for _ in range(num_palabras):
        # Pre-procesamiento del prompt
        prompt_proc = preproc_texto(prompt)
        prompt_proc = tokenizer.texts_to_sequences([prompt_proc])[0]
        prompt_proc = pad_sequences([prompt_proc], maxlen=max_sequence_len-1, padding='pre')

        # Prediccion de la siguiente palabra
        predict = model.predict(prompt_proc, verbose=0)
        predicted = np.argmax(predict, axis=1)

        # Concatenamos la palabra generada al prompt
        siguiente_palabra = ""
        for palabra,index in tokenizer.word_index.items():
            if index == predicted:
                siguiente_palabra = palabra
                break
        prompt += " " + siguiente_palabra
    return prompt.title()

In [ ]:
print(generar_texto("La Ciberseguridad", 8, model, max_sequence_len))

La Ciberseguridad En La Era Digital Protegiendo La Informacion Sensible


In [ ]:
print(generar_texto("La Inteligencia Artificial", 3, model, max_sequence_len))

La Inteligencia Artificial En La Educacion


In [ ]:
print(generar_texto("El futuro", 7, model, max_sequence_len))

El Futuro De La Educacion Impulsado Por La Ia


In [ ]:
print(generar_texto("El cambio climatico", 5, model, max_sequence_len))

El Cambio Climatico Sigue Siendo Una Amenaza Global


In [ ]:
print(generar_texto("La nueva ley", 3, model, max_sequence_len))

La Nueva Ley Y La Sostenibilidad


In [ ]:
print(generar_texto("El mundo", 6, model, max_sequence_len))

El Mundo De La Educacion Impulsado Por La
